In [1]:
!pip install numpy pandas torch scikit-learn matplotlib seaborn scikit-uplift causalml optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: pathos
    Found existing installation: pathos 0.3.2
    Uninstalling pathos-0.3.2:
      Successfully uninstalled pathos-0.3.2


In [ ]:
# General
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, os
import seaborn as sns
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

seed = 42

cuda:0


# 0. Prepare data

## 0.1. Split men & women dataset

In [3]:
# Data
from sklift.datasets import fetch_hillstrom
import pandas as pd

In [4]:
hillstrom_dataset = fetch_hillstrom()
full_data, full_target, full_treatment = fetch_hillstrom(target_col="visit",return_X_y_t=True)
pd.concat([full_data, full_target, full_treatment],axis=1).to_csv("./hillstrom_full_data_raw.csv", index=False)

mask_women = full_treatment != 'Womens E-Mail'
men_data_raw = full_data[mask_women]
men_target_raw = full_target[mask_women]
men_treatment_raw = full_treatment[mask_women]

# Xử lý biến Treatment (QUAN TRỌNG)
# Không dùng get_dummies ở đây vì dễ nhầm lẫn. Dùng map để kiểm soát 0 và 1.
# Mens E-Mail -> 1 (Treatment)
# No E-Mail   -> 0 (Control)
men_treatment_binary = men_treatment_raw.map({
    'Mens E-Mail': 1,
    'No E-Mail': 0
})
men_data = pd.get_dummies(men_data_raw, drop_first=True)
men_data_np, men_target_np, men_treatment_np = men_data.values.astype("float32"), men_target_raw.values.astype("float32"), men_treatment_binary.values.astype("float32")

# Women
mask_women = full_treatment != 'Mens E-Mail'
women_data_raw = full_data[mask_women]
women_target_raw = full_target[mask_women]
women_treatment_raw = full_treatment[mask_women]

women_treatment_binary = women_treatment_raw.map({
    'Womens E-Mail': 1,
    'No E-Mail': 0
})


women_data = pd.get_dummies(women_data_raw, drop_first=True)
women_data_np, women_target_np, women_treatment_np = women_data.values.astype("float32"), women_target_raw.values.astype("float32"), women_treatment_binary.values.astype("float32")

print("Splited data into women and men datasets")
print(f"Men: {men_data_np.shape}")
print(f"Women: {women_data_np.shape}")

Hillstrom dataset:   0%|          | 0.00/443k [00:00<?, ?iB/s]

Splited data into women and men datasets
Men: (42613, 15)
Women: (42693, 15)


## 0.2. Split train-set dataset & feature scaling

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split train test sets
X_men_train, X_men_test, y_men_train, y_men_test, t_men_train, t_men_test = train_test_split(
    men_data_np, men_target_np, men_treatment_np, test_size=0.2, random_state=seed, stratify=men_treatment_np
)

X_women_train, X_women_test, y_women_train, y_women_test, t_women_train, t_women_test = train_test_split(
    women_data_np, women_target_np, women_treatment_np, test_size=0.2, random_state=seed, stratify=women_treatment_np
)
print("Splited men & women datasets into train & test sets")
print(f"MEN - Train size: {X_men_train.shape[0]}, Test size: {X_men_test.shape[0]}")
print(f"WOMEN - Train size: {X_women_train.shape[0]}, Test size: {X_women_test.shape[0]}\n")

# Features scaling
scaler = StandardScaler()
X_men_train_scaled = scaler.fit_transform(X_men_train)
X_men_test_scaled = scaler.transform(X_men_test)

X_women_train_scaled = scaler.fit_transform(X_women_train)
X_women_test_scaled = scaler.transform(X_women_test)
print("Feature scaled X_men_train, X_men_test, X_women_train, X_women_test")

Splited men & women datasets into train & test sets
MEN - Train size: 34090, Test size: 8523
WOMEN - Train size: 34154, Test size: 8539

Feature scaled X_men_train, X_men_test, X_women_train, X_women_test


# 1. End to end pipeline

In [ ]:
from model.end_to_end_pipeline import Dragonnet

# TPE

In [ ]:
from sklift.metrics import uplift_auc_score

In [7]:
def objective(trial, X, y, t, device):
    # --- 1. Define Search Space theo Table 7 của Paper  ---
    
    # Hidden size & Outcome size là hệ số nhân (multiplier) trên input_dim [cite: 843-844]
    hidden_ratio = trial.suggest_float("hidden_ratio", 0.5, 2.0)
    outcome_ratio = trial.suggest_float("outcome_ratio", 0.5, 2.0)
    
    # Alpha & Beta cho Loss function
    alpha = trial.suggest_float("alpha", 0.5, 1.5)
    beta = trial.suggest_float("beta", 0.5, 1.5)
    
    # Learning rate (log scale)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-3, log=True)
    
    # Batch size (Paper không ghi cụ thể range cho batch_size trong bảng, 
    # nhưng implied qua steps_per_epoch. Ta tune batch_size để ổn định)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    
    # Tính toán kích thước layer thực tế
    input_dim = X.shape[1]
    shared_hidden = int(input_dim * hidden_ratio)
    outcome_hidden = int(input_dim * outcome_ratio) # Paper dùng multiplier cho outcome layer [cite: 740, 844]

    # --- 2. Split Validation Data cho việc Tuning ---
    # Paper dùng nested CV, ở đây ta dùng 1 tập valid đơn giản để demo nhanh
    X_train, X_val, y_train, y_val, t_train, t_val = train_test_split(
        X, y, t, test_size=0.2, random_state=42, stratify=t
    )
    
    # --- 3. Khởi tạo Model ---
    model = Dragonnet(
        input_dim=input_dim,
        shared_hidden=shared_hidden,
        outcome_hidden=outcome_hidden,
        alpha=alpha,
        beta=beta,
        learning_rate=learning_rate,
        batch_size=batch_size,
        epochs=30, # Paper giới hạn 30 epochs cho DragonNet [cite: 837]
        device=device,
        loss_type='tarreg' 
    )
    
    # --- 4. Train Model ---
    # Tắt in log training để đỡ rối màn hình khi search
    try:
        model.fit(X_train, y_train, t_train, valid_perc=None) 
    except Exception as e:
        # Trường hợp gradient nổ hoặc lỗi, trả về giá trị cực thấp
        return -100.0

    # --- 5. Predict & Evaluate trên Validation Set ---
    y0_val, y1_val, t_pred, eps = model.predict(X_val)
    uplift_val = (y1_val - y0_val).flatten()
    
    # Paper tối ưu hóa metric AUUC 
    score = uplift_auc_score(
        y_val.flatten(), 
        uplift_val, 
        t_val.flatten()
    )
    
    return score

In [8]:
from sklift.metrics import uplift_auc_score

In [9]:
import optuna
# Thiết lập logging để theo dõi quá trình
optuna.logging.set_verbosity(optuna.logging.INFO)

# --- 1. Tuning cho MEN Dataset ---
print("=== Starting TPE Search for MEN dataset ===")
study_men = optuna.create_study(direction="maximize") # Maximize AUUC
study_men.optimize(
    lambda trial: objective(trial, X_men_train_scaled, y_men_train, t_men_train, device), 
    n_trials=50, # Paper giới hạn 50 steps [cite: 839]
    timeout=10000 # Paper giới hạn 10000 giây [cite: 839]
)

print("Best params for Men:", study_men.best_params)
print("Best AUUC for Men:", study_men.best_value)

# --- 2. Tuning cho WOMEN Dataset ---
print("\n=== Starting TPE Search for WOMEN dataset ===")
study_women = optuna.create_study(direction="maximize")
study_women.optimize(
    lambda trial: objective(trial, X_women_train_scaled, y_women_train, t_women_train, device), 
    n_trials=50, 
    timeout=10000
)

print("Best params for Women:", study_women.best_params)
print("Best AUUC for Women:", study_women.best_value)

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():
[I 2025-12-20 03:32:24,644] A new study created in memory with name: no-name-f3bb8045-e85c-4ecb-83c2-a7d8a12bb0ce


=== Starting TPE Search for MEN dataset ===
epoch: 0--------- train_loss: 44.1181
epoch: 1--------- train_loss: 24.4842
epoch: 2--------- train_loss: 17.6810
epoch: 3--------- train_loss: 15.8392
epoch: 4--------- train_loss: 15.5185
epoch: 5--------- train_loss: 15.4818
epoch: 6--------- train_loss: 15.4758
epoch: 7--------- train_loss: 15.4721
epoch: 8--------- train_loss: 15.4690
epoch: 9--------- train_loss: 15.4662
epoch: 10--------- train_loss: 15.4636
epoch: 11--------- train_loss: 15.4611
epoch: 12--------- train_loss: 15.4588
epoch: 13--------- train_loss: 15.4565
epoch: 14--------- train_loss: 15.4544
epoch: 15--------- train_loss: 15.4523
epoch: 16--------- train_loss: 15.4503
epoch: 17--------- train_loss: 15.4483
epoch: 18--------- train_loss: 15.4465
epoch: 19--------- train_loss: 15.4446
epoch: 20--------- train_loss: 15.4428
epoch: 21--------- train_loss: 15.4411
epoch: 22--------- train_loss: 15.4394
epoch: 23--------- train_loss: 15.4378
epoch: 24--------- train_loss:

[I 2025-12-20 03:33:42,918] Trial 0 finished with value: 0.006643530907282929 and parameters: {'hidden_ratio': 1.8709064008647918, 'outcome_ratio': 1.6385363790151277, 'alpha': 1.4479185992952628, 'beta': 0.952336679210578, 'learning_rate': 0.00032739657742032285, 'batch_size': 64}. Best is trial 0 with value: 0.006643530907282929.


epoch: 29--------- train_loss: 15.4287
epoch: 0--------- train_loss: 247.9212
epoch: 1--------- train_loss: 170.8840
epoch: 2--------- train_loss: 139.9353
epoch: 3--------- train_loss: 127.9074
epoch: 4--------- train_loss: 119.3794
epoch: 5--------- train_loss: 111.7866
epoch: 6--------- train_loss: 104.9473
epoch: 7--------- train_loss: 98.8527
epoch: 8--------- train_loss: 93.4848
epoch: 9--------- train_loss: 88.8148
epoch: 10--------- train_loss: 84.8173
epoch: 11--------- train_loss: 81.4701
epoch: 12--------- train_loss: 78.7310
epoch: 13--------- train_loss: 76.5322
epoch: 14--------- train_loss: 74.7960
epoch: 15--------- train_loss: 73.4476
epoch: 16--------- train_loss: 72.4187
epoch: 17--------- train_loss: 71.6478
epoch: 18--------- train_loss: 71.0809
epoch: 19--------- train_loss: 70.6716
epoch: 20--------- train_loss: 70.3810
epoch: 21--------- train_loss: 70.1776
epoch: 22--------- train_loss: 70.0367
epoch: 23--------- train_loss: 69.9392
epoch: 24--------- train_los

[I 2025-12-20 03:34:03,759] Trial 1 finished with value: 0.002889715566169725 and parameters: {'hidden_ratio': 0.5449592710700613, 'outcome_ratio': 0.5666507068273163, 'alpha': 0.913283329490975, 'beta': 1.2817625638741812, 'learning_rate': 0.0002072367769605585, 'batch_size': 256}. Best is trial 0 with value: 0.006643530907282929.


...

epoch: 29--------- train_loss: 17.7723
epoch: 0--------- train_loss: 302.8669
epoch: 1--------- train_loss: 198.3763
epoch: 2--------- train_loss: 163.2238
epoch: 3--------- train_loss: 134.3479
epoch: 4--------- train_loss: 112.1566
epoch: 5--------- train_loss: 95.9132
epoch: 6--------- train_loss: 84.5373
epoch: 7--------- train_loss: 76.9074
epoch: 8--------- train_loss: 72.0106
epoch: 9--------- train_loss: 69.0093
epoch: 10--------- train_loss: 67.2570
epoch: 11--------- train_loss: 66.2847
epoch: 12--------- train_loss: 65.7726
epoch: 13--------- train_loss: 65.5163
epoch: 14--------- train_loss: 65.3937
epoch: 15--------- train_loss: 65.3368
epoch: 16--------- train_loss: 65.3102
epoch: 17--------- train_loss: 65.2969
epoch: 18--------- train_loss: 65.2892
epoch: 19--------- train_loss: 65.2836
epoch: 20--------- train_loss: 65.2790
epoch: 21--------- train_loss: 65.2748
epoch: 22--------- train_loss: 65.2708
epoch: 23--------- train_loss: 65.2670
epoch: 24--------- train_loss:

[I 2025-12-20 05:31:17,778] Trial 48 finished with value: 0.03416612391726324 and parameters: {'hidden_ratio': 1.9163919039796782, 'outcome_ratio': 1.7659243073001303, 'alpha': 0.7119480206301829, 'beta': 1.3238794295547551, 'learning_rate': 0.00047840255983126593, 'batch_size': 256}. Best is trial 44 with value: 0.041845136573858605.


epoch: 29--------- train_loss: 65.2459
epoch: 0--------- train_loss: 343.9238
epoch: 1--------- train_loss: 143.3413
epoch: 2--------- train_loss: 57.7990
epoch: 3--------- train_loss: 26.2836
epoch: 4--------- train_loss: 17.9244
epoch: 5--------- train_loss: 16.4701
epoch: 6--------- train_loss: 16.3120
epoch: 7--------- train_loss: 16.2972
epoch: 8--------- train_loss: 16.2928
epoch: 9--------- train_loss: 16.2896
epoch: 10--------- train_loss: 16.2868
epoch: 11--------- train_loss: 16.2840
epoch: 12--------- train_loss: 16.2810
epoch: 13--------- train_loss: 16.2780
epoch: 14--------- train_loss: 16.2748
epoch: 15--------- train_loss: 16.2717
epoch: 16--------- train_loss: 16.2686
epoch: 17--------- train_loss: 16.2656
epoch: 18--------- train_loss: 16.2628
epoch: 19--------- train_loss: 16.2602
epoch: 20--------- train_loss: 16.2579
epoch: 21--------- train_loss: 16.2557
epoch: 22--------- train_loss: 16.2531
epoch: 23--------- train_loss: 16.2505
epoch: 24--------- train_loss: 16

[I 2025-12-20 05:32:32,019] Trial 49 finished with value: 0.040818548793308894 and parameters: {'hidden_ratio': 1.5281801219351223, 'outcome_ratio': 1.5920697652233244, 'alpha': 0.9932007807019695, 'beta': 1.2309880510960134, 'learning_rate': 0.0008424905312730603, 'batch_size': 64}. Best is trial 44 with value: 0.041845136573858605.


epoch: 29--------- train_loss: 16.2375
Best params for Women: {'hidden_ratio': 1.5679455286745405, 'outcome_ratio': 1.6483456869142235, 'alpha': 0.953854620029144, 'beta': 1.3739164009436546, 'learning_rate': 0.0005996383425775317, 'batch_size': 64}
Best AUUC for Women: 0.041845136573858605


# Graph